In [4]:
!uv pip install ultralytics
import ultralytics
ultralytics.checks()

Ultralytics 8.3.230 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 38.0/107.7 GB disk)


In [ ]:
!unzip /content/coco128.zip

In [ ]:
!unzip /content/test.zip

In [9]:
from ultralytics import YOLO

# Load a model
model = YOLO('yolo11n.pt')

# Use the model
results = model.train(data='/content/coco128.yaml', epochs=500)  # train the model
results = model.val()  # evaluate model performance on the validation set

Ultralytics 8.3.230 🚀 Python-3.12.12 torch-2.9.0+cu126 CPU (Intel Xeon CPU @ 2.20GHz)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=16, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/coco128.yaml, degrees=0.0, deterministic=True, device=cpu, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=500, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolo11n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0,

AttributeError: partially initialized module 'polars' has no attribute '_cpu_check' (most likely due to a circular import)

In [ ]:
!yolo predict model=/content/runs/detect/train/weights/best.pt source='/content/test' save=True

In [ ]:
!pip install ultralytics deep-sort-realtime opencv-python

In [ ]:
from ultralytics import YOLO
import cv2
from deep_sort_realtime.deepsort_tracker import DeepSort
import numpy as np
import random
import colorsys


# =========================================
# 1. Load YOLO model
# =========================================
model = YOLO("/content/runs/detect/train5/weights/best.pt")


# =========================================
# 2. Class names (13 class của bạn)
# =========================================
class_names = [
    "Ryoma", "Teeme", "Zata", "Marja", "Baldum", "Hayate",
    "Keera", "Wisp", "Lorion", "Zephys",
    "Minions", "Slayer minion", "Moving slayer"
]


# =========================================
# 3. Random Color cho từng CLASS
#    (phiên bản màu sáng đẹp hơn HSV)
# =========================================
def bright_color():
    h = random.random()
    s = 0.8
    v = 0.95
    r, g, b = colorsys.hsv_to_rgb(h, s, v)
    return int(r * 255), int(g * 255), int(b * 255)

# Gán màu cho từng class
color_map = {cls_name: bright_color() for cls_name in class_names}


# =========================================
# 4. DeepSORT tracker
# =========================================
tracker = DeepSort(max_age=20)


# =========================================
# 5. Load video
# =========================================
cap = cv2.VideoCapture("/content/minimap_cut.mp4")


while True:
    ret, frame = cap.read()
    if not ret:
        break

    # YOLO detect
    results = model(frame)[0]

    # Xử lý YOLO -> detections
    detections = []
    for box in results.boxes:
        x1, y1, x2, y2 = map(int, box.xyxy[0])
        conf = float(box.conf)
        cls  = int(box.cls)

        detections.append(([x1, y1, x2, y2], conf, cls))

    # UPDATE TRACKER PHẢI NẰM TRONG VÒNG LẶP
    tracks = tracker.update_tracks(detections, frame=frame)

    # DUYỆT TRACKS
    for track in tracks:
        if not track.is_confirmed():
            continue

        x1, y1, x2, y2 = track.to_tlbr()
        track_id = track.track_id
        cls_id   = track.det_class
        conf     = track.det_conf

        cls_name = class_names[cls_id]
        color    = class_colors[cls_id]

        cv2.rectangle(frame, (x1,y1), (x2,y2), color, 2)
        cv2.putText(frame,
                    f"ID {track_id} | {cls_name} {conf:.2f}",
                    (x1, y1-5),
                    cv2.FONT_HERSHEY_SIMPLEX,
                    0.5,
                    color,
                    2)

    # # =====================================
    # # Hiển thị (cho Google Colab)
    # # =====================================
    # from google.colab.patches import cv2_imshow
    # cv2_imshow(frame)


cap.release()